In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-06-20 08:06:19,131 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Tweets 150 / 100 /skip gram

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet150-100-skip.model')
model.init_sims(replace=True)

2017-06-20 08:06:19,757 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet150-100-skip.model
2017-06-20 08:06:33,622 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet150-100-skip.model.wv.* with mmap=None
2017-06-20 08:06:33,623 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet150-100-skip.model.wv.syn0.npy with mmap=None
2017-06-20 08:06:36,685 : INFO : setting ignored attribute syn0norm to None
2017-06-20 08:06:36,688 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet150-100-skip.model.syn1neg.npy with mmap=None
2017-06-20 08:06:59,503 : INFO : setting ignored attribute cum_table to None
2017-06-20 08:06:59,538 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet150-100-skip.model
2017-06-20 08:07:08,320 : INFO : precomputing L2-norms of word weight vectors


In [3]:
print(model.most_similar(positive=['rss', 'haha']))

[(u'hehe', 0.933465838432312), (u'hehehe', 0.9157791137695312), (u'rs', 0.9122323393821716), (u'hahah', 0.9093583226203918), (u'haaha', 0.894985556602478), (u'kk', 0.8939653038978577), (u'heheh', 0.8911392688751221), (u'hahaha', 0.886341392993927), (u'rsrs', 0.879311740398407), (u'kkk', 0.8659310340881348)]


In [4]:
print(model.most_similar(positive=[':D', ':/']))

[(u'^^', 0.8823074102401733), (u':))', 0.8732483386993408), (u':(', 0.8686283826828003), (u':P', 0.8680444955825806), (u':)', 0.865323543548584), (u':DD', 0.8553106784820557), (u':p', 0.8550288677215576), (u';D', 0.8503556251525879), (u':3', 0.8495063781738281), (u'(:', 0.8468736410140991)]


In [5]:
print(model.most_similar(positive=['fofo', 'cute']))

[(u'fofinho', 0.8604645729064941), (u'meigo', 0.817651629447937), (u'fofis', 0.8074029684066772), (u'mordivel', 0.8007888793945312), (u'mord\xedvel', 0.79353928565979), (u'lindinho', 0.7911279201507568), (u'apert\xe1vel', 0.784753680229187), (u'fofinhoo', 0.7831857204437256), (u'fooofo', 0.7829524278640747), (u'foooofo', 0.7823171615600586)]


In [6]:
num_features= 100

In [7]:
df = pd.read_csv('/home/bahbbc/personality-normalized-word2vec.csv', encoding='utf-8')
df.shape

(1039, 186)

In [8]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [9]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [10]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

In [11]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [12]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [13]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [14]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-06-20 08:08:06,281 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:06,613 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:06,621 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:07,224 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:07,454 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:07,805 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:08,477 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:08,556 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:08,767 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:08,916 : WARNING : cannot compute similarity with no input []
2017-06-20 08:08:08,966 : WARNING : cannot compute similarity with no input []


CPU times: user 3.67 s, sys: 13.9 ms, total: 3.69 s
Wall time: 3.8 s


In [15]:
del model

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [17]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   21.4s finished


CPU times: user 4.48 s, sys: 315 ms, total: 4.79 s
Wall time: 27.8 s


In [18]:
joblib.dump(model, '/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model1.pkl')

['/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model1.pkl']

In [19]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['extraversion_m']

In [20]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,99,43
1,82,88


In [21]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.55      0.70      0.61       142
          1       0.67      0.52      0.58       170

avg / total       0.61      0.60      0.60       312



In [22]:
accuracy_score(yt, yp)

0.59935897435897434

In [23]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [24]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['extraversion_m']

In [25]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       387
          1       1.00      1.00      1.00       340

avg / total       1.00      1.00      1.00       727



In [26]:
print accuracy_score(ytt, ytp)

0.998624484182
